In [ ]:
# Look at Event Rates using Slices and CRT Header

In [ ]:
import ROOT
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import sys
import multiprocessing as mp
import uproot
import pandas as pd
import pickle
import h5py
import gzip
import math
import timeit

# First attempt
#infile = "../NTuples/eta_production_ntuple_test_nodirt.root"

# Latest Eta Production
#infile = "../NTuples/eta_production_nodirt_ntuple_batch1.root"

# MC CV Sample --> Low Stats
#infile = "../NTuples/mc_production_with_fmatch_ntuple.root"

#MC CV Sample --> High Stats
#infile = "../NTuples/mc_production_with_fmatch_ntuple_more_stats.root"
infile = "../NTuples/mc_production_with_fmatch_ntuple_more_stats_v2.root"
crt_header_file = "../NTuples/mc_production_crt_header_ntuple_v1.root"

In [ ]:
crtFile = uproot.open(crt_header_file)
inFile = uproot.open(infile)

inFileROOT = ROOT.TFile.Open(infile, "READ")
#h_tot_pot = inFileROOT.Get("TotalPOT")
h_tot_pot = inFileROOT.Get("TOTPOT_Clone")
TOT_POT = h_tot_pot.GetBinContent(1)
inFileROOT.Close()
TOT_POT = f"{TOT_POT:.2e}"
print("Total POT", TOT_POT)

#slc_tree = inFile["slc_truth_tree"]
slc_tree = inFile["slc_truth_tree"]
slc_reco_tree = inFile["slc_tree"]

slc_df = slc_tree.arrays(slc_tree.keys(), library="pd")
slc_reco_df = slc_reco_tree.arrays(slc_reco_tree.keys(), library="pd")


crt_h_tree = crtFile["crt_event_tree"]
crt_df = crt_h_tree.arrays(crt_h_tree.keys(), library="pd")

slc_df[:2]

In [ ]:
def isTPC(row):
    zm = 509.4
    xm = 201.3
    ym = 203.73225
    x, y, z = -9999, -9999, 9999
    try:
        x = abs(row["vtx_x"])
        y = abs(row["vtx_y"])
        z = abs(row["vtx_z"])
    except:
        x, y, z = -9999, -9999, 9999
        
    if (x <= xm) and (y <= ym) and (0 <= z <= zm):
        return 1
    else:
        return 0

slc_df["inTPC"] = slc_df.apply(isTPC, axis=1)
slc_reco_df["inTPC"] = slc_reco_df.apply(isTPC, axis=1)
slc_df[:2]

In [ ]:
crt_df[:4]

In [ ]:
runs = slc_df["run"].values
subruns = slc_df["subrun"].values
evts = slc_df["evt"].values

data = {'run': runs,
        'subrun': subruns,
        'evt': evts}

event_df = pd.DataFrame(data)

event_df = event_df.drop_duplicates()

# Drop duplicates to find unique combinations
#unique_combinations = event_df.drop_duplicates()

# Get the number of unique combinations
N_EVENTS = event_df.shape[0]

print("N_EVENTS:", N_EVENTS)


In [ ]:
N_slc = slc_df.shape[0]

slc_df = slc_df.drop_duplicates()

N_slc_n = slc_df.shape[0]
print(N_slc)
print(N_slc_n)

In [ ]:
N_slc_r = slc_reco_df.shape[0]
slc_reco_df = slc_reco_df.drop_duplicates()
N_slc_r_n = slc_reco_df.shape[0]

print(N_slc_r)
print(N_slc_r_n)

In [ ]:
slc_df["fmatch_time"] = slc_reco_df["fmatch_time"]
slc_df[:2]

# Select Slices in Time with the Beam

In [ ]:
def inTime(row):
    t = row["fmatch_time"]
    if 0 <= t <= 2.1:
        return 1
    else:
        return 0

slc_df["inTime"] = slc_df.apply(inTime, axis=1)
slc_df[:2]

In [ ]:
slc_df = slc_df.query("inTime == 1")
slc_df[:2]

In [ ]:
av_df = slc_df.query("inTPC == 1")

dirt_df = slc_df.query("inTPC == 0")

av_df[:2]

In [ ]:
def get_av(row):
    r, sr, e = row["run"], row["subrun"], row["evt"]
    m1 = (av_df["run"].values == r)
    m2 = (av_df["subrun"].values == sr)
    m3 = (av_df["evt"].values == e)
    m = m1 & m2 & m3
    return np.sum(m)

event_df["N_av"] = event_df.apply(get_av, axis=1)
event_df[:3]

In [ ]:
def get_dirt(row):
    r, sr, e = row["run"], row["subrun"], row["evt"]
    m1 = (dirt_df["run"].values == r)
    m2 = (dirt_df["subrun"].values == sr)
    m3 = (dirt_df["evt"].values == e)
    m = m1 & m2 & m3
    return np.sum(m)

event_df["N_dirt"] = event_df.apply(get_dirt, axis=1)
event_df[:3]

In [ ]:

# Event Loop Section

h_2d_nu = ROOT.TH2D("h_2d_nu", "", 8, 0, 8, 8, 0, 8)


# Loop over events
for num in range(event_df.shape[0]):
    if num % 10000 == 0:
        print("Analyzing event", num)
        
    row = event_df.iloc[num]
    N_av = row["N_av"]
    N_dirt = row["N_dirt"]
    
    h_2d_nu.Fill(N_dirt, N_av)


   
print("Finished Event Loop")

In [ ]:
ROOT.gStyle.SetPalette(ROOT.kRainBow)
h_2d_nu.SetStats(0)
h_2d_nu.SetTitle("")

c = ROOT.TCanvas("c", "c", 700, 500)
h_2d_nu.GetXaxis().SetTitle("Number of Dirt #nu")
h_2d_nu.GetYaxis().SetTitle("Number of AV #nu")
#h_2d_nu.GetZaxis().Setlog()
ROOT.gStyle.SetPaintTextFormat("1.1e")
h_2d_nu.Draw("Colz TEXT")
ROOT.gPad.SetLogz()
c.Draw()

In [ ]:
def get_nu_rate(h):
    N_tot = 0
    N_nu = 0
    for i in range(1, h.GetNbinsX()+1):
        for j in range(1, h.GetNbinsY()+1):
            if j >= 2:
                N_nu += h.GetBinContent(i, j)
            N_tot += h.GetBinContent(i, j)
            
    return ((1.0*N_nu)/N_tot)


nu_r_d = get_nu_rate(h_2d_nu)
print(nu_r_d)

In [ ]:
def get_other_rate(h):
    N_tot = 0
    N_other = 0
    for i in range(1, h.GetNbinsX()+1):
        for j in range(1, h.GetNbinsY()+1):
            if i >= 2:
                N_other += h.GetBinContent(i, j)
            N_tot += h.GetBinContent(i, j)
            
    return ((1.0*N_other)/N_tot)


dirt_r = get_other_rate(h_2d_nu)


print(dirt_r)